# Simulate extraction mechanisms



## Example of software usage 

In this example notebook, we will reproduce Figure 2 of 

- Z. Zhang, J. Pawliszyn, Headspace solid-phase microextraction, Analytical chemistry 65 (1993) 1843–1852.

In [1]:
#First import numpy, since all arrays are based on numpy
import numpy as np 

#Import matplotlib, for plotting
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
#Import the main software library
import simex_lib as simex

All parameters must be adopted in the same units of space/time/mass/volume
here we adopt:
- space units: centimeters (cm) 
- time units: seconds (s)
- mass: Micrograms (mug)
- volume: cubic-centimeters or milliliters (cm^3, mL)


In [3]:
#Experiment basename (for filenames and figures)
name = "simex_example_zhang1993_fig2"

The first step is to set the domain, that is, the position of the interfaces and boundaries.

In [4]:
#Domain definition (position of interfaces) 
# x0   x1   x2  x3 
# |----|----|---|
#x=np.array([initial_position, interface_1, interface_2, final_position])
#x=np.array([0, 0.0056, 0.07, 0.1]) # cm 
x=np.array([0, 0.1-0.07, 0.1-0.0056, 0.1]) # cm  - invert device

Now we set the names of the compartments.

In [5]:
#Names of compartments
#xnames = ["", "", ""]
#xnames = ["coating", "headspace", "aqueous"]
xnames = ["aqueous", "headspace", "coating"] # inverted device

Set initial concentrations, constant in each compartment.

In [6]:
#Initial concentrations in each space
#  Assumed constant for each space
#   C0   C1   C2   
# |----|----|----|
#C=np.array([C0, C1, C2])   # (mug/mL)
#C=np.array([0.0, 0.0, 1.0]) # (mug/mL)
C=np.array([1.0, 0.0, 0.0]) # (mug/mL) - inverted device


Diffusion coefficients for each compartment, constant at each one.

In [7]:
#Diffusion coefficients for each space
#   D0   D1   D2   
# |----|----|----|
#D=np.array([D0, D1, D2])           # (cm^2/s)
#D=np.array([2.8e-6, 0.077, 1.8e-5]) # (cm^2/s) 
D=np.array([1.8e-5, 0.077, 2.8e-6]) # (cm^2/s) - inverted device

Set the partition coefficients. Always enforce zero at external boundaries for no flow condition.

In [8]:
#Partition coefficients K for each interface
# Set 0.0 for beggining and endpoints    
# K0  K1  K2   K3
# |---|---|----|
#K=np.array([K0, K1, K2, K3]) # 2 boundaries and 2 interface coefficient, non-dimensional
#K=np.array([0.0, 50, 0.2, 0.0]) # 
K=np.array([0.0, 1/0.2, 1/50, 0.0]) # inverted device

Now configure runtime options for this simulation.

In [9]:
#Max time definition (how long to run the simulation)
maxtime = 120 # seconds

#Time step size
dt = 0.001  # seconds - depending on your diffusion coefficients, this may be required to be small

#Plotting time spots (snapshots of the concentrations are saved in these time instants)
iplot_time=np.array([0.0, 3.0, 15.0, 30.0, 60.0, 120]) # seconds
#iplot_time=np.linspace(0, 50, 21, endpoint=True) #Equally spaced plots, for animations

#Space discretization (number of grid points) - depending on your device, this may need to be incresed
N = 500

We now build the device with these parameters (we have given the same names of the variables as the device function arguments, but we could have called the variables differently).

In [10]:
p = simex.device(
        D = D, 
        K = K, 
        C = C, 
        xspace = x, 
        xnames = xnames, 
        name = name, 
        N = N, dt = dt, maxtime = maxtime, iplot_time = iplot_time)


        --------------------------------------------------------------
        Simex - Simulation of Extraction Processes
        --------------------------------------------------------------
        
You defined a device with 3 compartment(s).
Mechanism layout/interfaces (x):  [0.     0.03   0.0944 0.1   ]
Initial concentrations: [1. 0. 0.]
Diffusion coefficients: [1.8e-05 7.7e-02 2.8e-06]
Interface coefficients: [0.   5.   0.02 0.  ]
Output basename: output/simex_example_zhang1993_fig2/simex_example_zhang1993_fig2

Compartment 0  setup
 Name:                   aqueous
 Local Domain:           [0.   0.03]
 Difusion (neigbours):   [0.0e+00 1.8e-05 7.7e-02]
 Border/Interfaces Coef: [0. 5.]

Compartment 1  setup
 Name:                   headspace
 Local Domain:           [0.03   0.0944]
 Difusion (neigbours):   [1.8e-05 7.7e-02 2.8e-06]
 Border/Interfaces Coef: [5.   0.02]

Compartment 2  setup
 Name:                   coating
 Local Domain:           [0.0944 0.1   ]
 Difusion (neigbou



Now that the device structure is built, we can actually use in a few ways. 

Lets loop in time the model to see how the concentrations evolve in time. The main function here is run(), which calculates all time steps of an implicit solver for the model of the device. It will save and return snapshots at all time set in iplot_time.

In [ ]:
concentrations = p.run()

 It:  0  Time:  0.0  Mass:  0.03000  %Dif Eq:  229.6000 %
 It:  3000  Time:  3.0  Mass:  0.03013  %Dif Eq:  227.2692 %
 It:  15000  Time:  15.0  Mass:  0.03004  %Dif Eq:  122.5575 %
 It:  30000  Time:  30.0  Mass:  0.03000  %Dif Eq:  47.8957 %


The snapshots were saved in the csv files and also returned to the list of snapshots given at "concentrations". We can use this output to plot the concentrations, as follows. 

In [ ]:
fig, axes = plt.subplots(1, 1, constrained_layout=True, figsize=(15,10))
plt.xlabel("x-distance (cm)")
plt.ylabel("Concentration ($\mu g/mL$)")
plt.title("Reproduction of Zhang and Pawliszyn Fig 2")

for i, c in enumerate(concentrations):    
    #Plot
    t = iplot_time[i]
    axes.plot(p.x, c, label=str(t)+"s")
    
for i, name in enumerate(xnames):
    xlabel=0.5*p.xspace[i]+0.5*p.xspace[i+1]
    plt.text(xlabel, -0.1, name)
    
axes.plot(p.x, p.u_equi_ext, 'k--', label="Theoretical \n Equilibrium")
axes.legend()
plt.savefig(p.basename+"_final.png")